# Drinking Task Pipeline


Diese Pipeline führt eine Klassifikation von Trinkbewegungen durch.

Die Elemente, welche angepasst werden können, um Ansätze auszuprobieren, wurden mit ✏️ gekennzeichnet.

#### Geeignete Parameter (Beispiel):

**3.2 Ansatz auswählen**
- Ansatz 1: Absolute Datenpunkte mit geschnittenen Videos
- n_frames: **38**
    
**4.3 Modell anwenden**
- n_neurons: **64**
- RNN-Modell

# 1. Import Dependecies

In [50]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import RandomizedSearchCV

import math

# 2. Landmarks extrahieren

In [51]:
def extract_landmarks_normal():
    %run extract_landmarks.py --video-dir data/video/normal --output-file data/landmarks_normal.csv

In [52]:
def extract_landmarks_compensation():
    %run extract_landmarks.py --video-dir data/video/compensation --output-file data/landmarks_compensation.csv

In [53]:
# Falls die Landmarks neu extrahiert werden sollen, folgende Befehle auskommentieren: ✏️
# extract_landmarks_normal()
# extract_landmarks_compensation()

# 3. Data Pre-Processing

In [54]:
# Landmarks einlesen
data_normal_original = pd.read_csv('data/landmarks_normal.csv')
data_compensation_original = pd.read_csv('data/landmarks_compensation.csv')

# Augmentierte Datenpunkte für Datenanalyse einlesen
data_normal_augmented = pd.read_csv('../../AvatarDataAugmentation/Data/KeyPointsBereinigtNichtKompensiert_03.csv')
data_compensation_augmented = pd.read_csv('../../AvatarDataAugmentation/Data/KeyPointsBereinigtKompensiert_01.csv')
data_compensation_augmented2 = pd.read_csv('../../AvatarDataAugmentation/Data/KeyPointsBereinigtKompensiert_02.csv')

# Zusätzliche aus Unity generierte Datenpuntke einlesen
data_normal_augmented_unity = pd.read_csv('../../AvatarDataAugmentation/Data/KeypointsExportKompensiert.csv')
data_compensation_augmented_unity = pd.read_csv('../../AvatarDataAugmentation/Data/KeypointsExportKompensiert.csv')

data_normal = pd.concat([data_normal_original, data_normal_augmented, data_normal_augmented_unity], ignore_index=True) 
data_compensation = pd.concat([data_compensation_original, data_compensation_augmented2, data_compensation_augmented_unity], ignore_index=True) 

# Label setzen (Kompensation = 1, keine Kompensation = 0)
data_compensation.compensation = 1
data_normal.compensation = 0

data = pd.concat([data_compensation, data_normal], axis=0)
data[data.frame == 1]

,path,frame,compensation,x_0,y_0,z_0,x_1,y_1,z_1,x_2,...,z_29,x_30,y_30,z_30,x_31,y_31,z_31,x_32,y_32,z_32
0,data/video/compensation\01_trinken_kompensatio...,1,1,1000.09450,295.08030,-434.73780,1016.1854,265.61652,-381.51593,1025.4126,...,-142.551970,952.20350,1076.1581,-163.268000,1083.4438,1195.5463,-371.589500,882.91846,1209.9233,-408.50876
239,data/video/compensation\02_trinken_kompensatio...,1,1,1004.91460,272.73907,-510.62988,1019.1108,241.79927,-459.76190,1028.6346,...,-329.010470,894.49110,1262.0361,-357.484380,1198.8416,1341.8446,-638.522160,827.46140,1330.2144,-690.30240
529,data/video/compensation\03_trinken_kompensatio...,1,1,1020.42883,267.42682,-428.96457,1037.8142,237.42938,-372.71088,1048.0687,...,-39.149357,917.36487,1188.9327,-127.484695,1046.4462,1207.4360,-247.560130,865.76776,1265.4349,-387.39243
828,data/video/compensation\04_trinken_kompensatio...,1,1,1028.70000,259.41913,-509.90570,1046.9830,230.39180,-457.48190,1057.3713,...,171.714160,920.50530,1136.3898,-14.946421,1048.7849,1185.5167,-0.154483,862.41925,1285.5558,-266.17584
1174,data/video/compensation\05_trinken_kompensatio...,1,1,1026.52210,290.28644,-331.21103,1041.6088,261.87927,-267.49520,1051.1260,...,-132.645080,915.38214,1191.8250,-372.592470,992.2608,1183.2690,-359.144100,862.46747,1262.1813,-689.38250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64581,/Users/salomekoller/Library/CloudStorage/OneDr...,1,0,1071.85000,322.31950,-286.50940,0.0000,0.00000,0.00000,0.0000,...,0.000000,0.00000,0.0000,0.000000,1124.9250,1121.7290,-296.097400,1043.86500,1122.5790,-284.53160
64787,/Users/salomekoller/Library/CloudStorage/OneDr...,1,0,1092.69400,330.29700,-399.08220,0.0000,0.00000,0.00000,0.0000,...,0.000000,0.00000,0.0000,0.000000,1143.7100,1129.6740,-412.846700,1062.85000,1130.4050,-399.40930
64994,/Users/salomekoller/Library/CloudStorage/OneDr...,1,0,1109.26300,504.19490,-467.08040,0.0000,0.00000,0.00000,0.0000,...,0.000000,0.00000,0.0000,0.000000,1199.6830,1139.6010,-368.618400,1120.19600,1139.6920,-348.77690
65199,/Users/salomekoller/Library/CloudStorage/OneDr...,1,0,1149.92300,501.24380,-400.91390,0.0000,0.00000,0.00000,0.0000,...,0.000000,0.00000,0.0000,0.000000,1204.8050,1301.5190,-372.496500,1126.71000,1301.8360,-347.29240


## 3.1 Data Centering

In [55]:
""" 
Zentrierung Datenpunkte für Datenanalyse 
Dies ist die Zentrierung des Avatars am Kopf, indem dieser zum Punkt (0,0,0) gebracht wird. 
Alle anderen Keypoints werden entsprechend diesen X, Y und Z Werten verschoben. 

Die Zentrierung wird anstelle des von Nils erstellten Ansatzes verwendet, 
damit die Daten für die Datenanalyse im späteren Schritt verwendet werden können.

"""
x_cols = []
y_cols = []
z_cols = []
for i in range(33):
    x_cols.append(f'x_{i}')
    y_cols.append(f'y_{i}')
    z_cols.append(f'z_{i}')


In [56]:
videos_raw = list(data.groupby(data.path))
len(videos_raw)

602

In [ ]:
def center_keypoints(uncentered_videos):

    centred_df = pd.DataFrame()
    for path, video in uncentered_videos:
        keypoints = video[x_cols + y_cols + z_cols]

        # Step 2: get the head position and use it as cetner of mass
        center_of_mass = keypoints[['x_0', 'y_0', 'z_0']].iloc[0].values
        #print(center_of_mass)

        # Step 3: Translate keypoints
        tmp = pd.DataFrame()
        tmp[x_cols] = keypoints[x_cols] - center_of_mass[0]
        tmp[y_cols] = keypoints[y_cols] - center_of_mass[1]
        tmp[z_cols] = keypoints[z_cols] - center_of_mass[2]
        tmp.insert(0, 'path', path)
        tmp.insert(0, 'frame', video['frame'])
        tmp.insert(0, 'compensation', video['compensation'])
  
        centred_df = pd.concat([centred_df, tmp], axis=0, ignore_index=True)

    return centred_df
df_centred = center_keypoints(videos_raw)

In [58]:
df_centred.head(5)

,compensation,frame,path,x_0,x_1,x_2,x_3,x_4,x_5,x_6,...,z_23,z_24,z_25,z_26,z_27,z_28,z_29,z_30,z_31,z_32
0,1,1,/Users/salomekoller/Library/CloudStorage/OneDr...,0.0000,-1001.265,-1001.265,-1001.265,-1001.265,-1001.265,-1001.265,...,7.371407,7.371387,9.394707,9.399187,-6.115406,-6.115967,-5.740273,-5.740273,47.437397,47.436827
1,1,2,/Users/salomekoller/Library/CloudStorage/OneDr...,-3.3216,-1001.265,-1001.265,-1001.265,-1001.265,-1001.265,-1001.265,...,-2.696713,-3.498616,-29.445363,-13.956810,-63.243623,-64.332943,-5.740273,-5.740273,-14.759285,-16.102833
2,1,3,/Users/salomekoller/Library/CloudStorage/OneDr...,1.6550,-1001.265,-1001.265,-1001.265,-1001.265,-1001.265,-1001.265,...,-8.997166,-10.653107,-45.717673,-27.773403,-97.982793,-99.005583,-5.740273,-5.740273,-52.912333,-53.810943
3,1,4,/Users/salomekoller/Library/CloudStorage/OneDr...,3.9860,-1001.265,-1001.265,-1001.265,-1001.265,-1001.265,-1001.265,...,-10.049377,-11.663836,-44.307243,-28.706163,-97.977963,-98.740323,-5.740273,-5.740273,-52.726213,-53.393883
4,1,5,/Users/salomekoller/Library/CloudStorage/OneDr...,5.4320,-1001.265,-1001.265,-1001.265,-1001.265,-1001.265,-1001.265,...,-10.798463,-12.359276,-44.008033,-29.568633,-98.200573,-98.692723,-5.740273,-5.740273,-52.941793,-53.249863


## 3.2 Export centered Data for Data Analysis

### 3.2.1 Export Datenset "Kompensiert"

In [59]:
df_centred_augmented = df_centred[df_centred.path.str.startswith('/Users/')]
df_centred_original = df_centred[df_centred.path.str.startswith('data/')]

df_centred_augmented_compensated = df_centred_augmented[df_centred_augmented['compensation'] == 1]
df_centred_augmented_compensated.head(5)

,compensation,frame,path,x_0,x_1,x_2,x_3,x_4,x_5,x_6,...,z_23,z_24,z_25,z_26,z_27,z_28,z_29,z_30,z_31,z_32
0,1,1,/Users/salomekoller/Library/CloudStorage/OneDr...,0.0000,-1001.265,-1001.265,-1001.265,-1001.265,-1001.265,-1001.265,...,7.371407,7.371387,9.394707,9.399187,-6.115406,-6.115967,-5.740273,-5.740273,47.437397,47.436827
1,1,2,/Users/salomekoller/Library/CloudStorage/OneDr...,-3.3216,-1001.265,-1001.265,-1001.265,-1001.265,-1001.265,-1001.265,...,-2.696713,-3.498616,-29.445363,-13.956810,-63.243623,-64.332943,-5.740273,-5.740273,-14.759285,-16.102833
2,1,3,/Users/salomekoller/Library/CloudStorage/OneDr...,1.6550,-1001.265,-1001.265,-1001.265,-1001.265,-1001.265,-1001.265,...,-8.997166,-10.653107,-45.717673,-27.773403,-97.982793,-99.005583,-5.740273,-5.740273,-52.912333,-53.810943
3,1,4,/Users/salomekoller/Library/CloudStorage/OneDr...,3.9860,-1001.265,-1001.265,-1001.265,-1001.265,-1001.265,-1001.265,...,-10.049377,-11.663836,-44.307243,-28.706163,-97.977963,-98.740323,-5.740273,-5.740273,-52.726213,-53.393883
4,1,5,/Users/salomekoller/Library/CloudStorage/OneDr...,5.4320,-1001.265,-1001.265,-1001.265,-1001.265,-1001.265,-1001.265,...,-10.798463,-12.359276,-44.008033,-29.568633,-98.200573,-98.692723,-5.740273,-5.740273,-52.941793,-53.249863


In [60]:
df_centred_augmented_compensated.to_csv('../Data/KeypointsBereinigtKompensiert.csv',
                           sep=",",
                           encoding='utf-8',
                           index=False) 

### 3.2.2 Export Datenset "Nicht Kompensiert"

In [61]:
df_centred_augmented_notCompensated = df_centred_augmented[df_centred_augmented['compensation'] == 0]
df_centred_augmented_notCompensated.head(5)

,compensation,frame,path,x_0,x_1,x_2,x_3,x_4,x_5,x_6,...,z_23,z_24,z_25,z_26,z_27,z_28,z_29,z_30,z_31,z_32
1558,0,1,/Users/salomekoller/Library/CloudStorage/OneDr...,0.0000,-1001.265,-1001.265,-1001.265,-1001.265,-1001.265,-1001.265,...,7.371407,7.371387,9.394707,9.399187,-6.115406,-6.115967,-5.740273,-5.740273,47.437397,47.436827
1559,0,2,/Users/salomekoller/Library/CloudStorage/OneDr...,-3.3216,-1001.265,-1001.265,-1001.265,-1001.265,-1001.265,-1001.265,...,-2.696713,-3.498616,-29.445363,-13.956810,-63.243623,-64.332943,-5.740273,-5.740273,-14.759285,-16.102833
1560,0,3,/Users/salomekoller/Library/CloudStorage/OneDr...,1.6550,-1001.265,-1001.265,-1001.265,-1001.265,-1001.265,-1001.265,...,-8.997166,-10.653107,-45.717673,-27.773403,-97.982793,-99.005583,-5.740273,-5.740273,-52.912333,-53.810943
1561,0,4,/Users/salomekoller/Library/CloudStorage/OneDr...,3.9860,-1001.265,-1001.265,-1001.265,-1001.265,-1001.265,-1001.265,...,-10.049377,-11.663836,-44.307243,-28.706163,-97.977963,-98.740323,-5.740273,-5.740273,-52.726213,-53.393883
1562,0,5,/Users/salomekoller/Library/CloudStorage/OneDr...,5.4320,-1001.265,-1001.265,-1001.265,-1001.265,-1001.265,-1001.265,...,-10.798463,-12.359276,-44.008033,-29.568633,-98.200573,-98.692723,-5.740273,-5.740273,-52.941793,-53.249863


In [62]:
df_centred_augmented_notCompensated.to_csv('../Data/KeypointsBereinigtNichtKompensiert.csv',
                           sep=",",
                           encoding='utf-8',
                           index=False) 

In [63]:
df_centred_augmented = df_centred_augmented[['path', 'frame' ,'compensation', 'x_0', 'y_0', 'z_0', 'x_11', 'y_11', 'z_11',
           'x_12', 'y_12', 'z_12', 'x_13', 'y_13', 'z_13', 'x_14', 'y_14', 'z_14', 'x_15', 'y_15', 'z_15',
           'x_16', 'y_16', 'z_16', 'x_17', 'y_17', 'z_17', 'x_18', 'y_18', 'z_18', 'x_19', 'y_19', 'z_19',
           'x_20', 'y_20', 'z_20', 'x_21', 'y_21', 'z_21', 'x_22', 'y_22', 'z_22', 'x_23', 'y_23', 'z_23',
           'x_24', 'y_24', 'z_24', 'x_25', 'y_25', 'z_25', 'x_26', 'y_26', 'z_26', 'x_27', 'y_27', 'z_27',
           'x_28', 'y_28', 'z_28', 'x_31', 'y_31', 'z_31', 'x_32', 'y_32', 'z_32']] 

df_centred_original = df_centred_original[['path', 'frame' ,'compensation', 'x_0', 'y_0', 'z_0', 'x_11', 'y_11', 'z_11',
           'x_12', 'y_12', 'z_12', 'x_13', 'y_13', 'z_13', 'x_14', 'y_14', 'z_14', 'x_15', 'y_15', 'z_15',
           'x_16', 'y_16', 'z_16', 'x_17', 'y_17', 'z_17', 'x_18', 'y_18', 'z_18', 'x_19', 'y_19', 'z_19',
           'x_20', 'y_20', 'z_20', 'x_21', 'y_21', 'z_21', 'x_22', 'y_22', 'z_22', 'x_23', 'y_23', 'z_23',
           'x_24', 'y_24', 'z_24', 'x_25', 'y_25', 'z_25', 'x_26', 'y_26', 'z_26', 'x_27', 'y_27', 'z_27',
           'x_28', 'y_28', 'z_28', 'x_31', 'y_31', 'z_31', 'x_32', 'y_32', 'z_32']]

In [64]:
# Videos Original gruppieren und in Liste schreiben
videos_raw_augmented = list(df_centred_augmented.groupby(df_centred_augmented.path))
len(videos_raw_augmented)

# Videos Augmentiert gruppieren und in Liste schreiben
videos_raw_original = list(df_centred_original.groupby(df_centred_original.path))
len(videos_raw_original)

599

## 3.3 Pre-Processing-Funktionen

In [65]:
'''
Diese Funktion bringt die rohen Videos in eine neue Form, ohne die Videos zu schneiden.
Sie wird für den Sliding Window Ansatz benötigt.
Aus den rohen Videos werden nur noch die exrahierten Datenpunkte als Liste zurückgegeben.
'''
def remap_raw_videos(unmapped_videos):
    remapped_videos = []
    for video in unmapped_videos:
        v = video[1].reset_index()
        mapped_vid = v.loc[:, 'x_0':]
        remapped_videos.append(mapped_vid)
    return remapped_videos

In [66]:
'''
Schneiden der Videos

Diese Funktion bringt die rohen Videos in eine neue Form und schneidet sie eine bestimmte Anzahl Frames
vor und hinter dem vertikalen Höhepunkt der trinkenden Hand ab.

Aus den rohen Videos werden nur noch die exrahierten Datenpunkte als Liste zurückgegeben.

Inputs:
uncutted_videos: Die ungeschnittenen rohen Videos
n_frames: Die Anzahl der Frames, welche vor und nach dem Höchstpunkt der Hand (y_16) abgeschnitten werden sollen
'''
def cut_videos(uncutted_videos, n_frames=0):
    cut_videos = []
    for idx, video in enumerate(uncutted_videos):
        v = video[1].reset_index()
        # Position des höchsten Punktes (tiefster Wert, da tiefere Werte höhere vertikale Positionen aussagen)
        minpos = np.argmin(v.y_16)
        
        # Falls der Höhepunkt der Hand zu nahe am Beginn / Ende des Videos ist, ist die Trinkbewegung vermutlich nicht korrekt.
        # Das Video soll dann optimalerweise gelöscht werden.
        if((minpos < 20) or (len(v) - minpos < 20)):
            print('\033[91m' + 'Video mit dem Namen\n'
                  + videos_raw[idx][0]
                  + '\nund Index\n' + str(idx)
                  + '\nzeigt keine korrekte Trinkbewegung. Bitte entfernen.')
            
        cut_vid = v.loc[minpos-n_frames:minpos+n_frames, 'x_0':]   # Video x Frames vor und x Frames nach Höhepunkt der Hand (y_16) abschneiden
        cut_videos.append(cut_vid)
    return cut_videos

In [67]:
'''
Videos zentrieren

Diese Funktion zentriert alle Videos, in dem von allen Landmarks die Position des Kopfes im ersten Frame abgezogen wird.
'''
def center_data(uncentered_videos):
    centered_videos = []
    for video in uncentered_videos:
        centered_video = []
        # Position des Kopfes im ersten Frame des Videos bestimmen (x, y und z-Koordinate)
        head_start = video.loc[:,'x_0':'z_0'].values[0]
        for frame in np.array(video):
            centered_frame = []
            # Frame reshapen, sodass alle Landmarks als eine Liste zählen
            landmarks = frame.reshape((21, 3))
            for landmark in landmarks:
                centered_frame.append(landmark - head_start)
            centered_video.append(list(np.array(centered_frame).flatten()))
        centered_videos.append(centered_video)
    return centered_videos

In [68]:
'''
Relative Abstände

Diese Funktion berechnet den Abstand jedes Punktes des Skeletts zum Kopf und gibt diese anschliessend zurück.

Shape der Rückgabe: [x Anzahl Videos, x Anzahl Frames, 33 Datenpunkte]
'''
def calc_distances(raw_videos):
    # Abstand zu Kopf
    distances = []

    for video in raw_videos:
        frame_distances = []
        for frame in np.array(video):
            points = frame.reshape((33, 3))
            point_distances = []
            for k in range(len(points)):
                # Distanz einzeln von x-, y- und z-Koordinaten
                distance = points[k]-points[0]
                # Distanz mittels Formel berechnen
                point_distances.append(math.sqrt(distance[0] ** 2 + distance[1] ** 2 + distance[2] ** 2))
            frame_distances.append(point_distances)
        distances.append(frame_distances)
    return distances

In [69]:
'''
Diese Funktion gibt die Labels der Videos zurück.
1 = Compensation
0 = Natural
'''
def define_labels():
    labels = []

    for i in range(len(videos_raw_original)):
        labels.append(np.mean(videos_raw_original[i][1].compensation))
        
    for i in range(len(videos_raw_augmented)):
        labels.append(np.mean(videos_raw_augmented[i][1].compensation))
        
    return labels
labels = define_labels()

In [70]:
'''
Sliding Windows

Diese Funktion erstellt für alle Videos von unslided_videos Sliding Windows und gibt sie zurück.
Ausserdem werden die Labels auf die Sliding Windows korrekt verteilt.

Shape der Rückgabe: [x Anzahl Sliding Windows, {window_size} Anzahl Frames, 99 Datenpunkte]
'''
def create_sliding_windows(unslided_videos, window_size):
    videos_slided = []
    unslided_labels = define_labels()
    slided_labels = []
    for idx, unslided_video in enumerate(unslided_videos):
        video_label = unslided_labels[idx]
        for i in range(len(unslided_video) - window_size + 1):
            videos_slided.append(unslided_video[i:i+window_size])
            slided_labels.append(video_label)
    return videos_slided, slided_labels

In [71]:
'''
In dieser Funktion werden die Daten in eine geeignete Form gebracht.
Es werden die Values der einzelnen Videos in eine Liste geschrieben und zurückgegeben.
'''
def reshape_videos(unshaped_videos):
    reshaped_videos = []
    labels = []
    for video in unshaped_videos:
        reshaped_videos.append(video.values)
    return reshaped_videos

## 3.4 Ansatz auswählen ✏️

In [72]:
# Ansatz 1: Absolute Datenpunkte mit geschnittenen Videos
# videos = reshape_videos(cut_videos(videos_raw, n_frames=38))

# Ansatz 2: Zentrierte Datenpunkte mit geschnittenen Videos
# videos = center_data(cut_videos(videos_raw, n_frames=10))

# Ansatz 3: Relative Datenpunkte (Abstände zum Kopf) mit geschnittenen Videos
# videos = calc_distances(cut_videos(videos_raw, n_frames=10))

# Ansatz 4a: Sliding Windows mit absoluten Datenpunkten
videos_original, labels_original = create_sliding_windows(center_data(remap_raw_videos(videos_raw_original)), 10)
videos_augmented, labels_augmented = create_sliding_windows(center_data(remap_raw_videos(videos_raw_augmented)), 10)

# Ansatz 4b: Sliding Windows mit zentrierten Datenpunkten
# videos, labels = create_sliding_windows(center_data(remap_raw_videos(videos_raw)), 10)

# Ansatz 4b: Sliding Windows mit relativen Datenpunkten
# videos, labels = create_sliding_windows(calc_distances(remap_raw_videos(videos_raw)), 10)

np.array(videos_original).shape
np.array(videos_augmented).shape

(5741, 10, 63)

In [73]:
'''
Falls es zu einem Fehler kommt, kann folgende Zeile angepasst und auskommentiert werden, um ein falsches Video zu entfernen.
Alternativ kann das Video im Ordner gelöscht werden und die Datenpunkte neu extrahiert werden. ✏️
'''
# videos_raw.remove(videos_raw[144])

'\nFalls es zu einem Fehler kommt, kann folgende Zeile angepasst und auskommentiert werden, um ein falsches Video zu entfernen.\nAlternativ kann das Video im Ordner gelöscht werden und die Datenpunkte neu extrahiert werden. ✏️\n'

## 3.5 Features und Labels definieren

In [74]:
# Features bestimmen
X = np.asarray(videos_original)
X.shape

(142748, 10, 63)

In [75]:
# Labels setzen
y = np.array(labels_original)

y.shape

(142748,)

In [76]:
# Features bestimmen
X_train_salome = np.asarray(videos_augmented)
X_train_salome.shape

(5741, 10, 63)

In [77]:
# Labels setzen
y_train_salome = np.array(labels_augmented)

y_train_salome.shape

(5741,)

In [78]:
# Daten in ein Trainings- und Testset unterteilen
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=4)

In [79]:
""" 
Augmentierte Daten werden dem Trainingsset hinzugefügt 
"""
# Concatenate the additional data with the existing X_train and y_train
X_train = np.concatenate((X_train, X_train_salome), axis=0)
y_train = np.concatenate((y_train, y_train_salome), axis=0)

# Ensure the shape of X_train and y_train matches after concatenation
assert X_train.shape[0] == y_train.shape[0]

# Optionally, shuffle the concatenated data
indices = np.arange(X_train.shape[0])
np.random.shuffle(indices)
X_train = X_train[indices]
y_train = y_train[indices] 

In [80]:
# Daten in ein Trainings- und Testset unterteilen
X_validate, X_test, y_validate, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify=y_test, random_state=2)

In [81]:
# Verteilung der Testdaten
np.mean(y_test)

0.5771260449259795

# 4. Modelle trainieren

## 4.1 LSTM

In [82]:
def define_lstm_model(input_shape, n_neurons=1):
    lstm_model = keras.Sequential()
    lstm_model.add(layers.LSTM(n_neurons, activation='relu', input_shape=input_shape))
    lstm_model.add(layers.Dense(1, activation='sigmoid'))
    return lstm_model

## 4.2 RNN

In [83]:
def define_rnn_model(input_shape, n_neurons=1):
    rnn_model = keras.Sequential()
    rnn_model.add(layers.SimpleRNN(n_neurons, activation='relu', input_shape=input_shape))
    rnn_model.add(layers.Dense(1, activation='sigmoid'))
    return rnn_model

## 4.3 Modell anwenden

In [84]:
'''
Gewünschtes Modell erstellen

Auskommentieren und Anzahl Neuronen anpassen (n_neurons)
input_shape wird automatisch übernommen
'''
# Anzahl Neuronen bestimmen ✏️
n_neurons=64

# LSTM
# model = define_lstm_model(n_neurons=n_neurons, input_shape=X.shape[1:])

# RNN
model = define_rnn_model(n_neurons=n_neurons, input_shape=X.shape[1:])

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [85]:
# Modell kompilieren
model.compile(
    loss='binary_crossentropy',
    optimizer="Adam",
    metrics=['accuracy'],
)

In [86]:
X_train.shape

(105664, 10, 63)

In [ ]:
# Modell trainieren
model.fit(
    X_train, y_train, validation_data=(X_validate, y_validate), epochs=1000
)
# validation_data=(X_validate, y_validate),

In [88]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn_1 (SimpleRNN)        │ (None, 64)             │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,773 (96.77 KB)

 Trainable params: 8,257 (32.25 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 16,516 (64.52 KB)

### Modell speichern ✏️

In [89]:
# model_name = 'models/LSTM/lstm_a_cv_'
# model_name = 'models/RNN/rnn_a_cv_'
# model_name = 'models/sliding_window/sw_lstm_a_cv_'
model_name = 'models/sliding_window/sw_rnn_a_cv_'

# Wenn das Modell gespeichert werden soll, folgende Zeile auskommentieren:
# LEGACY
# model.save(model_name + str(X.shape[1]) + 'f_' + str(n_neurons) + 'n/' + 'model.h5')

model.save(model_name + str(X.shape[1]) + 'f_' + str(n_neurons) + 'n/' + 'my_model.keras')

## 4.4 Hyperparametertuning

In [90]:
'''
Hyperparametertuning

Diese Funktion findet die Anzahl an Neuronen, welche das beste Resultat liefert.
'''
def tune_hyperparameters(untuned_model):
    # Regressor mit dem mitgegebenen Modell erstellen
    keras_regressor = keras.wrappers.scikit_learn.KerasRegressor(untuned_model)

    # Eine Suche mit dem Regressor erstellen. Es werden Werte der Neuronen zwischen 1 und 128 ausgewertet.
    randomized_search_cv = RandomizedSearchCV(
        keras_regressor,
        {"n_neurons": np.arange(1, 128)},
        n_iter=10,
        cv=3
    )
    
    randomized_search_cv.fit(X_train, y_train, validation_data=(X_validate, y_validate), epochs=1000)
    
    print(randomized_search_cv.best_params_)
    
    return randomized_search_cv.best_estimator_.model

In [91]:
# Auskommentieren und Modell anpassen, um das beste Modell zu finden und zu speichern ✏️
# best_model = tune_hyperparameters(untuned_model=define_lstm_model)
# best_model.save('models/best_model')

# 5. Predictions

In [92]:
# Auskommentieren und ein bestimmtes Modell zu laden ✏️
# model = keras.models.load_model('models/RNN/rnn_a_cv_21f_32n')

In [93]:
result = model.predict(X_test)

result

670/670 ━━━━━━━━━━━━━━━━━━━━ 0s 506us/step


array([[1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       ...,
       [2.1799360e-03],
       [1.0000000e+00],
       [3.2184005e-04]], dtype=float32)

# 6. Evaluation

In [94]:
scores = model.evaluate(X_train, y_train, verbose=0)
print("In-Sample Accuracy: %.2f%%" % (scores[1]*100))
scores = model.evaluate(X_test, y_test, verbose=0)
print("Out-of-Sample Accuracy: %.2f%%" % (scores[1]*100))

In-Sample Accuracy: 99.35%
Out-of-Sample Accuracy: 99.24%


In [95]:
result = (result > 0.5).flatten()
actual = y_test > 0.5

In [96]:
confusion_matrix(actual, result)

array([[ 9001,    54],
       [  108, 12250]])

## Evaluation eines einzelnen Videos

In [97]:
# Gewünschtes Sliding-Window-Modell auskommentieren ✏️

model = keras.models.load_model('models/sliding_window/sw_rnn_a_cv_5f_64n/my_model.keras')
# model = keras.models.load_model('models/sliding_window/sw_lstm_a_cv_5f_64n')

# model = keras.layers.TFSMLayer('models/sliding_window/sw_rnn_a_cv_10f_64n')
# model = keras.models.load_model('models/sliding_window/sw_rnn_a_cv_77f_64n/my_model.keras')
# model = keras.models.load_model('models/sliding_window/sw_lstm_a_cv_10f_64n')
# model = keras.models.load_model('models/sliding_window/sw_rnn_a_cv_20f_64n')
# model = keras.models.load_model('models/sliding_window/sw_lstm_a_cv_20f_64n')

In [98]:
'''
Video klassifizieren

Diese Funktion berechnet für ein Video die Kompensationswahrscheinlichkeit

Input:
video_index: Index des Videos
window_size: Grösse der Sliding Windows

Output: Liste mit Wahrscheinlichkeiten für alle Sliding Windows der Videos
'''
def predict_video(video_index, window_size=5):
    return model.predict(np.array(create_sliding_windows(remap_raw_videos([videos_raw[video_index]]), window_size)[0]))